In [130]:
from docx import Document
import re
from icecream import ic

In [131]:
def load_docx(file_path):
    try:
        return Document(file_path)
    except Exception as e:
        print(f"Error loading document: {e}")
        return None

doc = load_docx('data/vb_test.docx')
if not doc:
    exit()


In [132]:
import string
seps = '[.,;-+]'
ascii = string.ascii_lowercase
lowest_level = [(i + ")") for i in ascii]
levels = [lowest_level]

In [133]:
def extract_text(doc):
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                full_text.append(cell.text)
    return '\n'.join(full_text)

document_text = extract_text(doc)


In [134]:
alphabet_lst = [i for i in ascii]
digit_lst = [str(i) for i in range(99)]
bullet_levels1 = [["chương"], ["điều"], digit_lst, alphabet_lst]
bullet_levels2 = [["chương"], ["phụ lục"], digit_lst, alphabet_lst]

In [135]:
from collections import OrderedDict

In [136]:
text = []
tree = OrderedDict()
for i, para in enumerate(doc.paragraphs):
    text.append(para.text)

In [137]:
full_text = []
c_check = False
for i, para in enumerate(text):
    if c_check == True:
        c_check = False
        continue
    if para.split(" ")[0].lower() == "chương":
        para = para + " " + text[i + 1]
        c_check = True
    else:
        c_check = False
    full_text.append(para)

In [138]:
def update_tree(bullet, par):
    check_text = True
    for i in range (len(bullet_levels1) - 1):
        if bullet.lower() in bullet_levels1[i]:
            check_text = False
            k = tree
            for j in range(i):
                k = k[next(reversed(k))]
            k[par] = OrderedDict()
    if check_text:
        k = tree
        while (k[next(reversed(k))] and list(k[next(reversed(k))].items())[0][0] != ""):
            k = k[next(reversed(k))]
        print("para", para)
        old_val = list(k[next(reversed(k))].items())
        val = ""
        if len(old_val) > 1:
            val = old_val[0][1]
        new_dict_val = val + par
        new_dict = {"": new_dict_val}
        k[next(reversed(k))] = OrderedDict(new_dict)

In [139]:
for i, para in enumerate(full_text):
    first_token = para.split(" ")[0]
    if len(first_token.strip()) > 0:
        bullet = re.split(r"[.,;)]",first_token)[0]
        update_tree(bullet, para)
    ic(tree)

ic| tree: OrderedDict([('Chương I NHỮNG QUY ĐỊNH CHUNG', OrderedDict())])
ic| tree: OrderedDict([('Chương I NHỮNG QUY ĐỊNH CHUNG',
                        OrderedDict([('Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng',
                                      OrderedDict())]))])
ic| tree: OrderedDict([('Chương I NHỮNG QUY ĐỊNH CHUNG',
                        OrderedDict([('Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng',
                                      OrderedDict([('1. Nghị định này quy định về Chứng '
                                                    'chỉ hành nghề dược; kinh doanh '
                                                    'dược; xuất khẩu, nhập khẩu thuốc; '
                                                    'đăng ký lưu hành dược liệu, tá '
                                                    'dược, vỏ nang; đánh giá cơ sở sản '
                                                    'xuất thuốc tại nước ngoài; thẩm '
                                                 

para Trong Nghị định này các từ, ngữ dưới đây được hiểu như sau:


                                                    'buổi giới thiệu thuốc hoặc thảo '
                                                    'luận chuyên đề khoa học liên quan '
                                                    'đến thuốc cho người hành nghề khám '
                                                    'bệnh, chữa bệnh.',
                                                    OrderedDict()),
                                                   ('3. Bán thành phẩm thuốc là sản phẩm '
                                                    'đã qua một, một số hoặc tất cả các '
                                                    'công đoạn chế biến, sản xuất, trừ '
                                                    'công đoạn đóng gói cuối cùng. ',
                                                    OrderedDict())]))]))])
ic| tree: OrderedDict([('Chương I NHỮNG QUY ĐỊNH CHUNG',
                        OrderedDict([('Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng',
                 

para Mục 1. HỒ SƠ, THỦ TỤC CẤP, CẤP LẠI, ĐIỀU CHỈNH NỘI DUNG VÀ THU HỒI CHỨNG CHỈ HÀNH NGHỀ DƯỢC


'
                                                    'bệnh, chữa bệnh.',
                                                    OrderedDict()),
                                                   ('3. Bán thành phẩm thuốc là sản phẩm '
                                                    'đã qua một, một số hoặc tất cả các '
                                                    'công đoạn chế biến, sản xuất, trừ '
                                                    'công đoạn đóng gói cuối cùng. ',
                                                    OrderedDict()),
                                                   ('4. Giá nhập khẩu của thuốc là trị '
                                                    'giá hải quan của thuốc nhập khẩu '
                                                    'được ghi trên tờ khai trị giá hải '
                                                    'quan tại cảng Việt Nam sau khi '
                                                    'thông quan.',
               

...

                       ('Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC',
                        OrderedDict([('',
                                      'd) Bản chính hoặc bản sao có chứng thực Giấy xác '
                                      'nhận thời gian thực hành theo quy định tại Mẫu số '
                                      '03 tại Phụ lục I ban hành kèm theo Nghị định này. '
                                      'Trường hợp thực hành tại nhiều cơ sở, thời gian '
                                      'thực hành được tính là tổng thời gian thực hành '
                                      'tại các cơ sở nhưng phải có Giấy xác nhận thời '
                                      'gian thực hành của từng cơ sở đó;')]))])
ic| tree: OrderedDict([('Chương I NHỮNG QUY ĐỊNH CHUNG',
                        OrderedDict([('Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng',
                                      OrderedDict([('1. Nghị định này quy định về Chứng '
                                          

para đ) Trường hợp đề nghị cấp Chứng chỉ hành nghề dược với phạm vi hoạt động khác nhau và yêu cầu thời gian thực hành, cơ sở thực hành chuyên môn khác nhau thì hồ sơ phải có Giấy xác nhận thời gian thực hành chuyên môn và nội dung thực hành chuyên môn của một hoặc một số cơ sở đáp ứng yêu cầu của mỗi phạm vi, vị trí hành nghề. Trường hợp các phạm vi hoạt động chuyên môn có cùng yêu cầu về thời gian thực hành và cơ sở thực hành chuyên môn thì không yêu cầu phải có Giấy xác nhận riêng đối với từng phạm vi hoạt động chuyên môn;
para e) Bản chính hoặc bản sao chứng thực giấy xác nhận kết quả thi do cơ sở tổ chức thi quy định tại khoản 2 Điều 28 của Nghị định này cấp đối với trường hợp Chứng chỉ hành nghề dược cấp theo hình thức thi;


tiếp cho người mua thuốc tại các cơ '
                                                    'sở bán lẻ thuốc.',
                                                    OrderedDict()),
                                                   ('9. Thặng số bán lẻ là trị giá tiền '
                                                    'chênh lệch giữa giá thuốc bán ra và '
                                                    'giá thuốc mua vào của cơ sở bán lẻ '
                                                    'thuốc.',
                                                    OrderedDict()),
                                                   ('10. Mức thặng số bán lẻ là tỉ lệ '
                                                    'phần trăm (%) giữa thặng số bán lẻ '
                                                    'và giá thuốc mua vào của cơ sở bán '
                                                    'lẻ thuốc.',
                                                    OrderedDict())]))])),
              

para g) Đối với người nước ngoài, người Việt Nam định cư ở nước ngoài đề nghị cấp Chứng chỉ hành nghề dược theo hình thức xét hồ sơ, phải có các tài liệu chứng minh về việc đáp ứng yêu cầu về sử dụng ngôn ngữ theo quy định tại khoản 2 Điều 14 của Luật dược.


TypeError: 'str' object does not support item assignment

In [ ]:
tree

OrderedDict([('Chương I NHỮNG QUY ĐỊNH CHUNG',
              OrderedDict([('Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng',
                            OrderedDict([('1. Nghị định này quy định về Chứng chỉ hành nghề dược; kinh doanh dược; xuất khẩu, nhập khẩu thuốc; đăng ký lưu hành dược liệu, tá dược, vỏ nang; đánh giá cơ sở sản xuất thuốc tại nước ngoài; thẩm quyền, hình thức, thủ tục thu hồi nguyên liệu làm thuốc, biện pháp xử lý nguyên liệu làm thuốc bị thu hồi; hồ sơ, trình tự thủ tục và thẩm quyền cấp giấy xác nhận nội dung thông tin, quảng cáo thuốc và biện pháp quản lý giá thuốc.',
                                          OrderedDict()),
                                         ('2. Nghị định này áp dụng đối với cơ quan, tổ chức, cá nhân trong nước và nước ngoài có hoạt động liên quan đến dược tại Việt Nam.',
                                          OrderedDict())])),
                           ('Điều 2. Giải thích từ ngữ ',
                            OrderedDict([('1. Thô

In [27]:
def flatten_tree(tree, parent_path="", separator=" > "):
    flat_list = []
    for key, value in tree.items():
        current_path = f"{parent_path}{separator}{key}" if parent_path else key
        # print(f"Processing: {current_path}")  # Debugging step
        if value:  # Check if the value is not blank OrderedDict
            flat_list.extend(flatten_tree(value, current_path, separator))
        else:
            # If value is a blank ordered dict, convert value to blank string
            flat_list.append((current_path, ""))
    return flat_list

In [28]:
flat_data = flatten_tree(tree)
chunks = [f"{path}: {text}" for path, text in flat_data]


In [ ]:
chunks

['Chương I NHỮNG QUY ĐỊNH CHUNG > Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng > 1. Nghị định này quy định về Chứng chỉ hành nghề dược; kinh doanh dược; xuất khẩu, nhập khẩu thuốc; đăng ký lưu hành dược liệu, tá dược, vỏ nang; đánh giá cơ sở sản xuất thuốc tại nước ngoài; thẩm quyền, hình thức, thủ tục thu hồi nguyên liệu làm thuốc, biện pháp xử lý nguyên liệu làm thuốc bị thu hồi; hồ sơ, trình tự thủ tục và thẩm quyền cấp giấy xác nhận nội dung thông tin, quảng cáo thuốc và biện pháp quản lý giá thuốc.: ',
 'Chương I NHỮNG QUY ĐỊNH CHUNG > Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng > 2. Nghị định này áp dụng đối với cơ quan, tổ chức, cá nhân trong nước và nước ngoài có hoạt động liên quan đến dược tại Việt Nam.: ',
 'Chương I NHỮNG QUY ĐỊNH CHUNG > Điều 2. Giải thích từ ngữ  > 1. Thông tin thuốc là việc thu thập, cung cấp các thông tin có liên quan đến thuốc bao gồm chỉ định, chống chỉ định, liều dùng, cách dùng, phản ứng có hại của thuốc và các thông tin khác liên quan đến ch

In [30]:
from langchain.vectorstores import FAISS
from langchain.embeddings import GPT4AllEmbeddings
from typing import List, Dict, Tuple
# Initialize the embedding model
embedding_model = GPT4AllEmbeddings(model_file="models/all-MiniLM-L6-v2-f16.gguf")

# Embed and store in FAISS
db = FAISS.from_texts(texts=chunks, embedding=embedding_model)

# Save the FAISS vector database
vector_db_path = "vectorstores/db_faiss"
db.save_local(vector_db_path)